[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mjavadpur/StyleHEAT3D/blob/main/quick_demo.ipynb)

In [ ]:
!git clone https://github.com/mjavadpur/StyleHEAT3D.git
%cd StyleHEAT3D


In [ ]:
# !pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 -f https://download.pytorch.org/whl/torch_stable.html
!pip install -r requirements.txt
!pip install gdown


In [ ]:
# print('Download pre-trained models...')
# !rm -rf checkpoints
# !bash /content/StyleHEAT3D/bash/download.sh


%mkdir checkpoints
!wget https://github.com/FeiiYin/StyleHEAT/releases/download/styleheat/Encoder_e4e.pth -O ./checkpoints/Encoder_e4e.pth
!wget https://github.com/FeiiYin/StyleHEAT/releases/download/styleheat/hfgi.pth -O ./checkpoints/hfgi.pth
!wget https://github.com/FeiiYin/StyleHEAT/releases/download/styleheat/StyleGAN_e4e.pth -O ./checkpoints/StyleGAN_e4e.pth
!wget https://github.com/FeiiYin/StyleHEAT/releases/download/styleheat/ffhq_pca.pt -O ./checkpoints/ffhq_pca.pt
!wget https://github.com/FeiiYin/StyleHEAT/releases/download/styleheat/ffhq_PCA.npz -O ./checkpoints/ffhq_PCA.npz
!wget https://github.com/FeiiYin/StyleHEAT/releases/download/styleheat/interfacegan_directions-20230323T133213Z-001.zip \
 -O ./checkpoints/interfacegan_directions-20230323T133213Z-001.zip
!unzip ./checkpoints/interfacegan_directions-20230323T133213Z-001.zip -d ./checkpoints/
!wget https://github.com/FeiiYin/StyleHEAT/releases/download/styleheat/stylegan2_d_256.pth -O ./checkpoints/stylegan2_d_256.pth
!wget https://github.com/FeiiYin/StyleHEAT/releases/download/styleheat/model_ir_se50.pth -O ./checkpoints/model_ir_se50.pth
!wget https://github.com/FeiiYin/StyleHEAT/releases/download/styleheat/StyleHEAT_visual.pt -O ./checkpoints/StyleHEAT_visual.pt
%mkdir ./checkpoints/Deep3D/
!wget -nc https://github.com/Winfredy/SadTalker/releases/download/v0.0.2/epoch_20.pth -O ./checkpoints/epoch_20.pth
# wget https://github.com/FeiiYin/StyleHEAT/releases/download/styleheat/epoch_20.pth -O ./checkpoints/epoch_20.pth
%mv checkpoints/epoch_20.pth checkpoints/Deep3D/epoch_20.pth
!wget https://github.com/Winfredy/SadTalker/releases/download/v0.0.1/BFM_Fitting.zip -O ./checkpoints/BFM_Fitting.zip
!unzip ./checkpoints/BFM_Fitting.zip -d ./checkpoints/BFM/
%mv ./checkpoints/BFM/BFM_Fitting/* ./checkpoints/BFM/
%rm -r ./checkpoints/BFM/BFM_Fitting

!wget https://github.com/FeiiYin/StyleHEAT/releases/download/styleheat/videos.zip -O ./checkpoints/videos.zip
!unzip ./checkpoints/videos.zip -d ./checkpoints/
%rm -rf ./checkpoints/__MACOSX
!rm ./checkpoints/videos.zip

%rm -rf /content/StyleHEAT/docs/demo/videos/
%rm -rf /content/StyleHEAT/docs/demo/audios/
!mkdir /content/StyleHEAT/docs/demo/
!mkdir /content/StyleHEAT/docs/demo/videos/
!mkdir /content/StyleHEAT/docs/demo/audios/
%mv ./checkpoints/videos/audios/* /content/StyleHEAT/docs/demo/audios/
%rm -rf ./checkpoints/videos/audios/
%mv ./checkpoints/videos/* /content/StyleHEAT/docs/demo/videos/

# pip install -i https://mirrors.cloud.tencent.com/pypi/simple pydub==0.25.1 yacs==0.1.8 librosa==0.6.0 numba==0.48.0 resampy==0.3.1 imageio-ffmpeg==0.4.7


In [ ]:
# Same-Identity Reenactment with a video.
!python /content/StyleHEAT3D/inference.py \
            --config /content/StyleHEAT3D/configs/inference.yaml \
            --video_source=/content/StyleHEAT3D/docs/demo/videos/RD_Radio34_003_512.mp4 \
            --output_dir=/content/StyleHEAT3D/docs/demo/output --if_extract

In [ ]:
# Cross-Identity Reenactment with a single image and a video.
# The --video_source and --image_source can be specified as either a single file or a folder.
# For a better inversion result but taking more time, 
# please specify --inversion_option=optimize and we will optimize the feature latent of StyleGAN-V2. 
# Otherwise we will use HFGI encoder to get the style code and inversion condition with --inversion_option=encode.
# If you need align (crop) images during the inference process, 
# please specify --if_align. Or you can first align the source images following FFHQ dataset.
# If you need to extract the 3dmm parameters of the target video during the inference process,
# please specify --if_extract. 
# Or you can first extract the 3dmm parameters with the script TODO.sh 
# and save the 3dmm in the {video_source}/3dmm/3dmm_{video_name}.npy
# If you only need to edit the expression without modifying the pose, 
# please specify --edit_expression_only.
!python inference.py \
            --config configs/inference.yaml \
            --video_source=./docs/demo/videos/RD_Radio34_003_512.mp4 \
            --image_source=./docs/demo/images/100.jpg \
            --cross_id --if_extract \
            --output_dir=./docs/demo/output

In [ ]:
# Intuitive Editing.
# The 3dmm parameters of the images can also be pre-extracted or online-extracted with the parameter --if_extract.
!python inference.py \
            --config configs/inference.yaml \
            --image_source=./docs/demo/images/40.jpg \
            --inversion_option=optimize \
            --intuitive_edit \
            --output_dir=./docs/demo/output \
            --if_extract

In [ ]:
# Attribute Editing.
# The support editable attributes include young, old, beard, lip.
# Note to preserve the editing attributes details in W space, 
# the optimized inversion method is banned here.
!python inference.py \
            --config configs/inference.yaml \
            --video_source=./docs/demo/videos/RD_Radio34_003_512.mp4 \
            --image_source=./docs/demo/images/40.jpg \
            --attribute_edit --attribute=young \
            --cross_id \
            --output_dir=./docs/demo/output

In [ ]:
# Audio Reenactment.
# Please first install SadTalker in the folder of third_part as the format of third_part/SadTalker. 
# Download its pre-trained checkpoints according to their instructions. 
# Install the additional libraries with pip install pydub==0.25.1 
# yacs==0.1.8 librosa==0.6.0 numba==0.48.0 resampy==0.3.1 imageio-ffmpeg==0.4.7. 
# Then you can run audio reenactment freely.
!python inference.py \
            --config configs/inference.yaml \
            --audio_path=./docs/demo/audios/RD_Radio31_000.wav \
            --image_source=./docs/demo/images/100.jpg \
            --cross_id --if_extract \
            --output_dir=./docs/demo/output \
            --inversion_option=optimize